# pygissim

A Tutorial

## What is pygissim?

pygissim is a library for running performance simulations on distributed computer systems. The author works in the area of Geographic Information Systems (GIS) and so the focus here is on Web GIS systems, but the underlying engine is generic and could be used to model many different types of systems that have jobs that are passed from node to node and back to the caller.

pygissim is in its early development, and so there may be need to inspect the code and see how it works. The API documentation is in /docs/_build/html/index.html

The fundamental concept:
1. Create (or load) a Design.
2. Add the Design to a Simulator.
3. Run the Simulator for a length of real-world time.
4. Analyze the results.

This tutorial will take you through each of these steps.

In [1]:
from pygissim.pygissim import *
import pandas as pd
import pygissim.nb as nb
import pygissim.util as util
import matplotlib.pyplot as plt

from yfiles_jupyter_graphs import GraphWidget
from typing import Dict, Tuple

## A Design

A Design contains other collections of objects and has ways to access them:
- Network zones
- Network connections
- Compute nodes
- Service providers
- Workflows

In [2]:
d: Design = Design(name="My Design", desc="An example")

## A Network

First, define network zones. They can be as simple as a single local network, or a web of zones that are interconnected. Zones do not have a specific bandwidth: bandwidth and latency are the job of Connections. A zone will have a local Connection, and Connections that connect it to other Zones.

In [3]:
lan: Zone = Zone(name="LAN", desc="A local zone", z_type=ZoneType.LOCAL)
dmz: Zone = Zone(name="DMZ", desc="The edge zone", z_type=ZoneType.EDGE)

Now, let's connect them. Each Zone will have its local Connection and there is a Connection in each direction from lan to dmz and from dmz to lan.

We can add the Zones and Connections to the Design

In [4]:
d.add_zone(lan, local_bw_mbps=1000, local_latency_ms=0)
d.add_zone(dmz, local_bw_mbps=1000, local_latency_ms=0)

# Note that the Connections need not be symmetrical
d.add_connection(lan.connect(dmz, bw=350, lat=1))
d.add_connection(dmz.connect(lan, bw=500, lat=1))

Now, let's visualize the network.

In [5]:
w:GraphWidget = nb.draw_network(d.zones, d.network)
w.show()

GraphWidget(layout=Layout(height='500px', width='100%'))

Note that each Zone is connected to itself (local connection) and has an incoming and outgoing Connection to the other Zone. This means the Zones are fully connected. Zones can connect in many different ways, including branching and looping.

pygissim comes with some template networks. You can list them and load them as shown below.

In [6]:
print(util.network_list)

['Local Only', 'Local and ArcGIS Online', 'Branch Offices', 'Cloudy', 'Backhaul Cloudy']


In [7]:
d.zones, d.network = util.load_network('Local and ArcGIS Online')
w= nb.draw_network(d.zones, d.network)
w.show()

GraphWidget(layout=Layout(height='500px', width='100%'))

### Knowledge Check
1. How many Connections does the Local network Zone have? (Hint: entry, exit and internal)
2. What is the bandwidth from the DMZ to the Internet?
3. What is the bandwidth from the Internet to the DMZ?
4. How many network Connections would a request have to traverse to go from Local to ArcGIS Online?

## Adding Compute

The building block for computation is the ComputeNode. They come in three types: clients, physical servers and virtual servers.

**Clients** are abstract and represent a type of device (workstation PC, mobile device) and are used to calculate service times, but not capacity. In real life, every added client to the system scales up the client tier.

**Physical servers** can either host software directly (see Service Providers) and/or be hosts for virtual servers.

**Virtual servers** are hosted by physical servers and have an allocation of CPU and memory from their host. They are used as Service Providers.

All ComputeNodes are located in a Zone.

Every ComputeNode has a HardwareDefinition that describes their overall capacity and performance. You can create a HardwareDefinition, or load one of the ones provided by pygissim.

In [8]:
lan: Optional[Zone] = d.get_zone('Local')
if lan is None: raise ValueError("Check that you have loaded one of the template networks with a 'Local' zone")

# Creating a Client "by hand"
client_hw = HardwareDef(processor='Intel Core i7-4770K', 
                        cores=4, specint_rate2017=20, 
                        architecture=ComputeArchitecture.INTEL, 
                        threading=ThreadingModel.PHYSICAL)
client = ComputeNode(name='Client PC', desc='Sample PC',
                     hw_def=client_hw,
                     memory_GB=16,
                     zone=lan,
                     type=ComputeNodeType.CLIENT)

# Using the LibManager to create a server. Check src/pygissim/data/hardware.csv for all pre-made types
lib = util.LibManager()
lib.load_local()
server_hw = lib.hardware['2018 Xeon Gold 6144']
server = ComputeNode(name='Local Server', desc='Physical server',
                     hw_def=server_hw,
                     memory_GB=128,
                     zone=lan,
                     type=ComputeNodeType.P_SERVER)

# Virtual machines hosted by the physical server
server.add_virtual_host(name="VMWeb", v_cores=4, memory_GB=8)
server.add_virtual_host(name="VMGISA", v_cores=4, memory_GB=16)
server.add_virtual_host(name="VMGISB", v_cores=4, memory_GB=16)
server.add_virtual_host(name="VMDS", v_cores=4, memory_GB=16)
server.add_virtual_host(name="VMSQL", v_cores=4, memory_GB=16)
server.add_virtual_host(name="VMFile", v_cores=4, memory_GB=16)

d.add_compute(client)
d.add_compute(server)

# Some compute power for ArcGIS Online
agol: Optional[Zone] = d.get_zone('ArcGIS Online');
if agol is None: raise ValueError("Check that you have loaded one of the template networks with a 'ArcGIS Online' zone")

agol_hw = lib.hardware['AMI db.r3.8xlarge (32vc)']
for i in range(1, 4):
    d.add_compute(ComputeNode(f'AGOL{i}','', agol_hw, 128, zone=agol, type=ComputeNodeType.P_SERVER))

Let's visualize the hardware as it relates to the network.
- Orange circles are Zones.
    - In this example, only the Local and ArcGIS Online Zones have ComputeNodes, so Internet and DMZ are unconnected (this isn't a network connectivity graph).
    - The Client PC and local server are in the Local Zone.
    - The AWS servers are in the ArcGIS Online Zone.
- The local physical has several hosted virtual servers.

In [9]:
w = nb.draw_zone_compute(d)
w.graph_layout = "organic"
w.show()

GraphWidget(layout=Layout(height='650px', width='100%'))

### Knowledge Check

1. How many virtual machines are hosted by the Local Server?
2. What Zone is the Client PC in?
3. How many physical ComputeNodes are in the ArcGIS Online Zone?
4. How many ComputeNodes are in the DMZ?

## Service Providers

Having ComputeNodes is useful, but until they are identified as a ServiceProvider, they won't do anything. Service types are simply a tag that represents a type of service being provided so that when a Workflow Definition says "portal", the system will know what ComputeNodes handle that type.

The types of services known by pygissim are defined in src/pygissim/data/services.csv

In [10]:
d.services = lib.service_definitions
nb.create_service_provider(d, "Web Browser", service="browser", node_names=['Client PC'])
nb.create_service_provider(d, "Pro Workstation", service="pro", node_names=['Client PC'])
nb.create_service_provider(d, "IIS", service="web", node_names=["VMWeb"])
nb.create_service_provider(d, "Portal", service="portal", node_names=["VMWeb"])
nb.create_service_provider(d, "Map Server", service="map", node_names=["VMGISA", "VMGISB"])
nb.create_service_provider(d, "Hosting Server", service="feature", node_names=["VMGISA", "VMGISB"])
nb.create_service_provider(d, "Data Store", service="relational", node_names=["VMDS"])
nb.create_service_provider(d, "SQL Server", service="dbms", node_names=["VMSQL"])
nb.create_service_provider(d, "File Server", service="file", node_names=["VMFile"])

# ArcGIS Online has a helper function to set up
nb.create_agol_service_providers(d, "ArcGIS Online")

To help visualize the relationships between ComputeNodes and ServiceProviders, let's draw them.

Hint: hover over the circles, and try playing with the different graph layouts.

- The gray circles are ServiceProviders.
    - In ArcGIS Online, three physical servers are sharing all of the ServiceProvider duties.
    - Locally, there is a division of Service types to individual virtual hosts.
    - The Hosting Server and Map Server have two load balanced virtual hosts, VMGISA and VMGISB.

In [11]:
# nb.compute_to_graph(d.compute_nodes())
w = nb.draw_compute_sp(d)
w.show()

GraphWidget(layout=Layout(height='760px', width='100%'))

### Knowledge Check

1. What ComputeNode is assigned the "Data Store" service?
2. What services does the Client PC provide?

## Workflows

Here's where the work happens. We need to define work being done by the system and how often it's happening.

As an example, if a user is working with ArcGIS Field Maps, they might have a Web Map loaded that has two layers in it, a base map and an overlay of points. The base map is an example of a cached **map** service and the overlay is an example of a **feature** service.

Making a cached **map** service request will start a chain of operations, with the request going first to the Web Adaptor, then the Portal, GIS Server, and then the File Server to get the tile(s). Then the result is passed up the chain and back to the mobile device. Each hop along the chain might need to traverse one or more network connections.

Setting up workflows "by hand" is a fair bit of code, so using the predefined ones in the library is useful.

In [12]:
mobile_wf_def: WorkflowDef = lib.workflow_definitions['Mobile Data Collection']
mobile: Workflow = Workflow('Mobile Workflow', '', type=WorkflowType.USER, definition=mobile_wf_def, user_count=10, productivity=4)

This pre-fab workflow definition sets up two operations that happen when the mobile user makes a request:
1. The mobile web feature layer and
2. The cached base map

These two "chains" of operations (WorkflowChain) can be completely divergent, accessing different servers and traversing different networks to solve the request. This complexity allows pygissim to simulate the parallel requests that are the norm for modern web maps. In this case, the requests are similar until they diverge, with one request going to a hosting server for feature data and the other going to a map service and getting map tiles.

In [13]:
w = nb.draw_workflows([mobile])
w.show()

GraphWidget(layout=Layout(height='640px', width='100%'))